# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [56]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np

In [57]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv('MAP_BOX_API')
px.set_mapbox_access_token(map_box_api)


# Import Data

In [58]:
# Import the necessary CSVs to Pandas DataFrames
census = pd.read_csv('Data/sfo_neighborhoods_census_data.csv', index_col='year')
coordinates = pd.read_csv('Data/neighborhoods_coordinates.csv')
housing = pd.read_csv('housing_units_by_year.csv')

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [70]:
# Define Panel Visualization Functions
def housing_units_per_year(): # Works
    """Housing Units Per Year."""
    df_info = pd.DataFrame(census.groupby(census.index).housing_units.mean())
    housing_units_per_year_plot = df_info.plot.bar(
        ylim = (df_info.housing_units.min() - df_info.housing_units.std(),
        df_info.housing_units.max() + df_info.housing_units.std()))
    return housing_units_per_year_plot


def average_gross_rent(): # Works
    """Average Gross Rent in San Francisco Per Year."""
    
    avgs = census.groupby(census.index)[['sale_price_sqr_foot','gross_rent']].mean()
    line2 = avgs.gross_rent.plot(c = 'red')
    title = plt.title('Average Gross Rent by Year')
    ylabel = plt.ylabel('Price Per SqFt')
    return line2,title,ylabel



def average_sales_price(): # Works
    """Average Sales Price Per Year."""
    avgs = census.groupby(census.index)[['sale_price_sqr_foot','gross_rent']].mean()
    line1 = avgs.sale_price_sqr_foot.plot(c = 'purple')
    title = plt.title('Average Price Per SqFt')
    ylabel = plt.ylabel('Price Per SqFt')
    return line1, title, ylabel



def average_price_by_neighborhood(): # Works
    """Average Prices by Neighborhood."""
    
    data = census[['neighborhood','sale_price_sqr_foot']].groupby([census.index, 'neighborhood']).mean().reset_index().dropna().sort_values(by ='year')
    hvplot1 = data.hvplot.line(x = 'year', y = 'sale_price_sqr_foot', groupby ='neighborhood', hover_cols = 'all')
    return hvplot


def top_most_expensive_neighborhoods(): # Works
    """Top 10 Most Expensive Neighborhoods."""

    expensive = pd.DataFrame(census.groupby(['neighborhood']).sale_price_sqr_foot.mean().sort_values(ascending = False).head(10).reset_index())
    expensive_plot = expensive.hvplot.bar(figsize = (30,20), x = 'neighborhood', rot = 90)
    return expensive_plot


def most_expensive_neighborhoods_rent_sales(): # Works
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    a = census.groupby(['year','neighborhood']).mean().drop(columns = 'housing_units')
    a = a.reset_index()
    plot1 = a.hvplot.bar(groupby = 'neighborhood', rot = 90, x = 'year', figsize = (30,10))
    return plot1

    
    
def parallel_coordinates(): # Works
    """Parallel Coordinates Plot."""
    new = census.reset_index()
    most_expensive = new.groupby('neighborhood').mean().sort_values(by = 'sale_price_sqr_foot', ascending = False).head(50)
    most_expensive['sale_price_sqr_foot'] = np.floor(most_expensive['sale_price_sqr_foot'])
    most_expensive['gross_rent'] = np.floor(most_expensive['gross_rent'])
    most_expensive = most_expensive.reset_index()

    most_expensive['year'] = round(most_expensive['year'])
    most_expensive = pd.DataFrame(most_expensive)

    pcoor = px.parallel_coordinates(
        most_expensive,
        dimensions = ['sale_price_sqr_foot', 'housing_units','gross_rent'],
        color = 'sale_price_sqr_foot',
        labels={'sale_price_sqr_foot':'Sale Price/SqFt', 'housing_units':'Housing Units', 'gross_rent':'Gross Rent'},
        width = 1400,
        height = 800)
    return pcoor

def parallel_categories(): # Works
    """Parallel Categories Plot."""
    
    new = census.reset_index()
    most_expensive = new.groupby('neighborhood').mean().sort_values(by = 'sale_price_sqr_foot', ascending = False).head(50)
    most_expensive['sale_price_sqr_foot'] = np.floor(most_expensive['sale_price_sqr_foot'])
    most_expensive['gross_rent'] = np.floor(most_expensive['gross_rent'])
    most_expensive = most_expensive.reset_index()

    most_expensive['year'] = round(most_expensive['year'])
    most_expensive = pd.DataFrame(most_expensive)

    pcat = px.parallel_categories(
        most_expensive,
        dimensions = ['neighborhood','year','housing_units','gross_rent'],
        color = 'sale_price_sqr_foot',
        labels={'neighborhood':'Neighborhood', 'sale_price_sqr_foot':'Sale Price/SqFt', 'housing_units':'Housing Units', 'gross_rent':'Gross Rent', 'year':'Year'},
        width = 1400,
        height = 800)
    return pcat



def neighborhood_map(): # Works
    """Neighborhood Map."""

    coordinates = pd.read_csv('Data/neighborhoods_coordinates.csv')
    coordinates.columns = ['neighborhood','lat','lon']
    coordinates = pd.DataFrame(coordinates)

    mean_df = pd.DataFrame(census.groupby(['neighborhood']).mean())
    new_df = pd.DataFrame(pd.merge(coordinates,mean_df, how = 'left', on = ['neighborhood','neighborhood']).dropna())
    map_box = px.scatter_mapbox(
        new_df,
        lat="lat",
        lon="lon",
        color="neighborhood",
        zoom=10)
    map_box.update_layout(
        mapbox_style="carto-positron")
     # Got this almost directly from one of the activities.
    return map_box.show()


def sunburst():
    """Sunburst Plot."""
    new = census.reset_index()
    most_expensive = new.groupby('neighborhood').mean().sort_values(by = 'sale_price_sqr_foot', ascending = False).head(50)
    most_expensive['sale_price_sqr_foot'] = np.floor(most_expensive['sale_price_sqr_foot'])
    most_expensive['gross_rent'] = np.floor(most_expensive['gross_rent'])
    most_expensive = most_expensive.reset_index()

    most_expensive['year'] = round(most_expensive['year'])
    most_expensive = pd.DataFrame(most_expensive)

    fig = px.sunburst(most_expensive, path = ['year','neighborhood',], values = 'sale_price_sqr_foot', width = 1400, height = 800)
    return fig


In [71]:
sunburst()

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# YOUR CODE HERE!


# Create a tab layout for the dashboard
# YOUR CODE HERE!


# Create the dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()